#**AI Ethics Bot**

##Gpt-2

We first start by downloading tensorflow 1.x and installing Gpt-2 simple.

In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
     |████████████████████████████████| 489.6 MB 16 kB/s 
     |████████████████████████████████| 1.3 MB 38.1 MB/s 
     |████████████████████████████████| 463 kB 38.9 MB/s 
     |████████████████████████████████| 5.8 MB 34.6 MB/s 


Now We check that the notebook is powered by the Gpu.

If no graphic card is running, go to the Colab menu above and select:

- Runtime --> Change runtime type --> GPU (in the "Hardware Accelerator" selection) --> Save

To make sure that Colab is actually setup to be supported by GPU computation, further check:

- Edit --> Notebook settings

In [ ]:
!nvidia-smi

Fri Nov 19 16:12:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


It is important to work already within the folder, beacuse Gpt-2 needs to be downloaded where the checkpoint runs. (this explains step [7] below):

In [ ]:
%cd /content/drive/Shareddrives/EthicsBot

/content/drive/Shareddrives/EthicsBot


The model has been fine-tuned already (on solely the Frankenstein text) and the current state "saved" as a checkpoint. When the checkpoint is saved on Drive is archived as a .tar file. It has to be unpacked.

In [ ]:
# This passage is not necessary, as the checkpoint models for "run1_frank", "run1_novels" and "run1_all" are already unpacked in the EthicsBot folder.
import tarfile
my_tar = tarfile.open('checkpoint_run1_novels.tar')
my_tar.extractall('')            # specify which folder to extract to (leave blank for same location)
my_tar.close()

Now we can download the operating model.

In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 209Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 6.62Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 341Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:09, 50.0Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 503Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 8.84Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 8.96Mit/s]


We start the session from the checkpoint and generate random text with no prompt. I have added a user input that allows to choose which model we want to load. Cell [7] below is the original cell and it works just fine by itself, but once a model is loaded it is not possible to try with another one without terminating the VM's process.

In [ ]:
#Skip to the next cell
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1_all')

***Important!*** By adding this cell below, we reset the graph of the model we have loaded (via input), allowing us to try different model without relaunching the entire Colab session!

In [ ]:
import tensorflow as tf
tf.compat.v1.reset_default_graph()  #This line is crucial to reset the graph/model. It uses this syntax because tf.reset is a deprecated function.
if not sess:
    sess = gpt2.start_tf_sess()
else:
    sess = gpt2.reset_session(sess)
model_input = input("Please choose one model checkpoint between run1_frank, run1_novels, run1_all : ")
gpt2.load_gpt2(sess, run_name=model_input)

Please choose run1_frank, run1_novels, run1_all : run1_frank
Loading checkpoint checkpoint/run1_frank/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1_frank/model-1000


In [ ]:
gpt2.generate(sess, run_name=model_input)

Six years have passed since I resolved on my present undertaking; and I still bear the relation of Elizabeth. She, who had been my constant companion and companion in school and society, was now engaged on a voyage to the utmost northern pole, and mightily armed with such materials as she hoped to procure. Her presence was regarded as an insurmountable burden to bear; but I, the miserable and helpless slave, resolved to assist her in her enterprise.

			It was on a dreary November night of November, that I beheld the accomplishment of my toils. With an anxiety that almost amounted to agony, I collected the instruments of life around me, that I might infuse a spark of being into the lifeless thing that lay at my feet. It was already one in the morning; the rain pattered dismally against the panes, and my candle was nearly burnt out, when, by the glimmer of the half-extinguished light, I saw the dull yellow eye of the creature open; it breathed hard, and a convulsive motion agitated its 

We are now ready to test our AI Ethics Bot by providing the prompt that we want (the attribute for Gpt-2 is "prefix" and not "prompt").

In [ ]:
gpt2.generate(sess,
              #run_name='run1_all',  #Old line with explicit checkpoint session, without user input
              run_name=model_input,
              length=250,
              temperature=0.7,  #default is 0.7, usually it is a decimal between 0.7 and 1.0
              prefix="ethical AI can be developed when",
              nsamples=5,
              batch_size=5
              )

ethical AI can be developed when the complexity of data sets is reduced and in a low-structured way, without necessarily causing significant harm. 
AI researchers should consider both the volume of data available and the quality of the data in which they are to be used, both in terms of the data itself and in terms of how they may be used. 
AI researchers should be aware that under certain circumstances, other data sets may not be available. For example, when using machine learning systems to train algorithms, researchers may lack access to data sets that are not relevant to the training algorithm. 
AI researchers should not use automated or series of automated testing methods. Instead, they should ensure that those methods meet the standards set by the relevant government and industry bodies.


AI Hints 
Chinese research agencies have established research programs using layered collections of machine learning data, and have identified examples of automated testing methods being applie

##TextgenRnn

It is important to *reset* the runtime because fot TextgenRnn it is necessary to run a different version of tensorflow.

In [ ]:
%tensorflow_version 2.x
!pip3 install git+git://github.com/minimaxir/textgenrnn.git#v1.5.0
from google.colab import files
from textgenrnn import textgenrnn
from datetime import datetime
import os

  Cloning git://github.com/minimaxir/textgenrnn.git to /tmp/pip-req-build-jh96slny
  Running command git clone -q git://github.com/minimaxir/textgenrnn.git /tmp/pip-req-build-jh96slny
  Created wheel for textgenrnn: filename=textgenrnn-2.0.0-py3-none-any.whl size=1734432 sha256=3ecdb3932797d0fee0ef09b1d70336ba434435f0aa7ce29e5f095a5cfe866ac5
  Stored in directory: /tmp/pip-ephem-wheel-cache-dhkkpym3/wheels/44/76/dd/267e0c4fa9fc4a76550510cd7913e258545176ccf6a2c34205
Successfully built textgenrnn


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


On the left bar, under "Files", wait until the icon-folder "drive" appears.
Edit the variables:
*   Change the "creativity" to get more or less randomness.
*   Change the "number" to control the number of generated responses.



In [ ]:
model1 = 'textgenrnn_weights_ai_principles_20.hdf5'
model2 = 'textgenrnn_weights_frank_words_10.hdf5'
creativity = 0.3
number = 2
prefix = "ethical AI can be developed when"
res = textgenrnn('/content/drive/Shareddrives/Rockwell RA Team /EthicsBot/Novel TM /AI Ethics Stuff/Models/char-rnn/' + 
                 model1).generate(number, temperature=creativity, prefix=prefix, return_as_list=True)
for row in res:
  print(row)

100%|██████████| 2/2 [00:23<00:00, 11.92s/it]

ethical AI can be developed when developed, including the risks of the concepts of the research and development of AI and a relevant standard of the safety and individuals are designed and accountable to ensure that the application of the discovery phase and development and information required to 
ethical AI can be developed when the development of data and the extent of the system and the company that is designed to develop and control over the development of AI and the interests of the control of the system to be able to assess the society and the right to access the system and responsible
